##  Steps
#### • Make a Data Frame with required columns DONE
#### • Make function that is called for each file DONE
#### • Extract The remaining two sections DONE
#### • Calculate the variables

In [2]:
import pandas as pd
import numpy as np
import seaborn as sbs
import matplotlib.pyplot as plt
from nltk import *
import urllib
import html2text
from bs4 import BeautifulSoup

In [3]:
stop_words = []
stop_words_file = open('StopWords_Generic.txt','r')
for word in stop_words_file:
    stop_words.append(word.lower().strip())


In [4]:
master_dictionary = pd.read_csv('master_dictionary.csv',usecols=['Word','Negative','Positive'])
master_dictionary['Word'] = master_dictionary['Word'].str.lower()
positive_words = master_dictionary[master_dictionary['Positive']!=0]['Word'].values
negative_words = master_dictionary[master_dictionary['Negative']!=0]['Word'].values


constraining_words = pd.read_excel('constraining_dictionary.xlsx')['Word'].str.lower().values
uncertain_words = pd.read_excel('uncertainty_dictionary.xlsx')['Word'].str.lower().values



In [5]:
url_prepend = 'https://www.sec.gov/Archives/'

In [6]:
cik_list = pd.read_excel('cik_list.xlsx')
columns = ['mda_positive_score',
 'mda_negative_score',
 'mda_polarity_score',
 'mda_average_sentence_length',
 'mda_percentage_of_complex_words',
 'mda_fog_index',
 'mda_complex_word_count',
 'mda_word_count',
 'mda_uncertainty_score',
 'mda_constraining_score',
 'mda_positive_word_proportion',
 'mda_negative_word_proportion',
 'mda_uncertainty_word_proportion',
 'mda_constraining_word_proportion',
 'qqdmr_positive_score',
 'qqdmr_negative_score',
 'qqdmr_polarity_score',
 'qqdmr_average_sentence_length',
 'qqdmr_percentage_of_complex_words',
 'qqdmr_fog_index',
 'qqdmr_complex_word_count',
 'qqdmr_word_count',
 'qqdmr_uncertainty_score',
 'qqdmr_constraining_score',
 'qqdmr_positive_word_proportion',
 'qqdmr_negative_word_proportion',
 'qqdmr_uncertainty_word_proportion',
 'qqdmr_constraining_word_proportion',
 'rf_positive_score',
 'rf_negative_score',
 'rf_polarity_score',
 'rf_average_sentence_length',
 'rf_percentage_of_complex_words',
 'rf_fog_index',
 'rf_complex_word_count',
 'rf_word_count',
 'rf_uncertainty_score',
 'rf_constraining_score',
 'rf_positive_word_proportion',
 'rf_negative_word_proportion',
 'rf_uncertainty_word_proportion',
 'rf_constraining_word_proportion',
 'constraining_words_whole_report']
variables = pd.DataFrame(columns=columns)

In [7]:
import re
WNlemma = WordNetLemmatizer()

### We dont Lemmatize words because the lists already include inflections of words

In [8]:

def complex_word(word):
    count = 0
    for letter in word:
        if letter in ['a','e','i','o','u']:
            count +=1
    if(word[-2:] in ['es','ed']):
        count -=1
        
    return count > 2


def read_sections(file,url):
    
    MDA_section = re.findall('\n\W*item[^a-z0-9]+\d[a-z]?\.?[^A-Za-z0-9]*management\Ws[^A-Za-z0-9]*discussion[^A-Za-z0-9]*and[^A-Za-z0-9]*analysis(.*?)\n\W*item[^a-z]+\d[a-z]?',file,flags=re.DOTALL)
    MDA_section.append('')
    MDA_section = max(MDA_section,key=len)
   
    QQDMR_section = re.findall('\n\W*item[^a-z]+\d[a-z]?\.?[^a-z0-9]*quantitative[^A-Za-z0-9]*and[^A-Za-z0-9]*qualitative[^A-Za-z0-9]*disclosure[s]?[^A-Za-z0-9]*about[^A-Za-z0-9]*market[^A-Za-z0-9]*risk(.*?)\n\W*item[^a-z]+\d[a-z]?',file,flags=re.DOTALL)
    QQDMR_section.append('')
    QQDMR_section = max(QQDMR_section,key=len)
    RF_section = re.findall('\n\W*item[^a-z]+\d[a-z]?\.?[^a-z0-9]*risk[^a-z0-9]*factors(.*?)\n\W*item[^a-z]+\d[a-z]?',file,flags=re.DOTALL)
    RF_section.append('')
    RF_section = max(RF_section,key=len)
    
    return (MDA_section,QQDMR_section,RF_section)
    
    

In [9]:
def calc_variables(section):
    sent_num = 0
    sents = sent_tokenize(section)
    
    words = [] 
    for sent in sents:
        Words = word_tokenize(sent)
        if(len(words)>2):
            sent_num +=1
        words+= [word for word in Words if word.isalpha() and word not in stop_words and len(word) > 1 and word in master_dictionary['Word'].values]

    if(sent_num==0):
        words=[]
    if len(words)!=0:  
          
        positive_score = len([word for word in words if word in positive_words])
        negative_score = len([word for word in words if word in negative_words])
        uncertainty_score = len([word for word in words if word in uncertain_words])
        constraining_score = len([word for word in words if word in constraining_words])
        positive_score = len([word for word in words if word in positive_words])
        word_count = len(words)
        polarity_score = (positive_score-negative_score)/(positive_score+negative_score+0.000001)
        subjectivity_score = (positive_score+negative_score)/(word_count+0.000001)
        average_sentence_length = word_count/sent_num
        complex_word_count = len([x for x in words if complex_word(x)])
        per_complex_words = complex_word_count/word_count
        pos_word_proportion = positive_score/word_count
        neg_word_proportion = negative_score/word_count
        uncer_word_proportion = uncertainty_score/word_count
        const_word_proportion = constraining_score/word_count
        fog_index = 0.4 * (average_sentence_length+per_complex_words)
        return[positive_score,negative_score,polarity_score,average_sentence_length,per_complex_words,fog_index,complex_word_count,word_count,uncertainty_score,constraining_score,pos_word_proportion,neg_word_proportion,uncer_word_proportion,const_word_proportion]
    else:
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    

In [22]:
ana_section = []
output_variables = []
for row_index, row in cik_list.iterrows():
    url = (url_prepend + row['SECFNAME'])
    response = urllib.request.urlopen(url)
    file =response.read().decode('utf-8').lower()
    if re.search('<html>',file):
        html2text.escape_all = True
        file = BeautifulSoup(file,'html5lib').text
    mda_section,qqdmr_section,rf_section =  read_sections(file,url)
    mda_variables = calc_variables(mda_section)
    qqdmr_variables = calc_variables(qqdmr_section)
    rf_variables = calc_variables(rf_section)
    const_whole_report = len([word for word in word_tokenize(file) if word in constraining_words])
    output_variables.append((mda_variables+qqdmr_variables+rf_variables+[const_whole_report]))
    print(row_index)
    
    
   
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151


In [23]:
output_df = cik_list.copy()
output_df = pd.concat([output_df,pd.DataFrame(output_variables,columns=columns)],axis=1,ignore_index=True,sort=False)
output_df.columns = list(cik_list.columns) + columns 

In [24]:
output_df.to_csv('output_variables.csv')